In [1]:
import sys
import os
root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_path)
import json
import logging
from pprint import pprint

from typing import List, Tuple, Dict
from collections import defaultdict
import math
from dataclasses import asdict

ROOT_DIR = r"C:\Projects\Research\SWEEP\SWEEP"

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

from src.common.models import ProcessModel, BPMNModel
from src.activity_breakdown import ActivityBreakdownGPT, ActivityBreakdownComparison, ActivityBreakdownMetrics, generate_activity_breakdown_metrics, compare_activity_breakdown

def get_sectors(train=True) -> List[str]:
    return [sector for sector in os.listdir("data") if os.path.isdir(os.path.join("data",f"{'train' if train else 'test'}", sector))]

def get_activities(sector: str, train=True) -> List[str]:
    sector_path = os.path.join("data",f"{'train' if train else 'test'}", sector)
    return [activity for activity in os.listdir(sector_path) if os.path.isdir(os.path.join(sector_path, activity))]\
        
def get_file_paths(sector: str, activity: str, model_name: str, train=True) -> Tuple[str, str, str, str]:
    dir_path = os.path.join(ROOT_DIR, "data", f"{'train' if train else 'test'}", sector, activity)
    test_path = os.path.join(ROOT_DIR, "test", "results", sector, activity)
    activity_breakdown_path = os.path.join(ROOT_DIR, dir_path, f"{activity}_activity_breakdown.json")
    bpmn_model_path = os.path.join(ROOT_DIR, dir_path, f"{activity}_model.txt")
    response_path = os.path.join(ROOT_DIR, test_path, f"{model_name}_response.json")
    return test_path, activity_breakdown_path, bpmn_model_path, response_path

def create_prompt_components_variations() -> List[Dict[str, str]]:
    variations = [
        {
            "name": "Neutral_Analyst_Basic",
            "components": None
        }
    ]
    
    return variations

def get_models() -> Dict[str, ActivityBreakdownGPT]:
    models = {}
    variations = create_prompt_components_variations()
    
    for variation in variations:
        model_name = f"GPT-3.5-Activity-Breakdown-{variation['name']}"
        model = ActivityBreakdownGPT(prompt_components=variation['components'])
        models[model_name] = model
    
    return models

def process_activity(sector: str, activity: str) -> None:
    logging.info(f"Processing sector: {sector}, activity: {activity}")
    
    try:
        # Load activity breakdown and ground truth
        _, activity_breakdown_path, bpmn_model_path, _ = get_file_paths(sector, activity, "dummy")
        bpmn_model = BPMNModel.from_text(bpmn_model_path)
        activity_breakdown_ground_truth = ProcessModel.from_json(activity_breakdown_path)
        
        models = get_models()
        
        for model_name, model in models.items():
            logging.info(f"Processing with model: {model_name}")
            
            test_path, _, _, response_path = get_file_paths(sector, activity, model_name)
            
            # Ensure test directory exists
            os.makedirs(test_path, exist_ok=True)
            
            # Get model response
            response = model.bpmn_activity_breakdown(bpmn_model.model)
            model_activity_breakdown = ProcessModel.from_dict(activity, response)
            
            # Compare results
            activity_breakdown_comparison: ActivityBreakdownComparison = compare_activity_breakdown(model_activity_breakdown, activity_breakdown_ground_truth)
            activity_breakdown_metrics: ActivityBreakdownMetrics = generate_activity_breakdown_metrics(activity_breakdown_comparison)
            
            response_dict = {
                "model": {
                    "name": model_name,
                    "components": model.prompt_components["_raw_input"]
                },
                "response": model_activity_breakdown.to_dict(),
                "metrics": activity_breakdown_metrics.to_dict()
            }
            
            # Save results
            with open(response_path, 'w') as f:
                json.dump(response_dict, f, indent=4)
            
            logging.info(f"Successfully processed and saved results for {sector}/{activity} with {model_name}")
    
    except Exception as e:
        logging.error(f"Error processing {sector}/{activity}: {str(e)}")

In [2]:
def main():
    start = True
    sectors = get_sectors()
    for sector in sectors:            
        if not start:
            continue          
        activities = get_activities(sector)
        for activity in activities:
            if not start:
                continue           
            process_activity(sector, activity)
            start = False

if __name__ == "__main__":
    main()

2024-09-05 23:34:52,290 - INFO - Processing sector: banking, activity: credit_application
2024-09-05 23:34:52,541 - INFO - Processing with model: GPT-3.5-Activity-Breakdown-Neutral_Analyst_Basic
2024-09-05 23:35:00,344 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-09-05 23:35:02,965 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-09-06 11:41:15,164 - ERROR - Error processing banking/credit_application: Activity mismatch: Appointment Decision vs Set Appointment
